In [1]:
from pathlib import Path
import polars as pl
from autofaiss import build_index
from FlagEmbedding import BGEM3FlagModel
from datasets import Dataset
import numpy as np
from core_pro.ultilities import make_dir
from item_matching import PipelineText

In [2]:
path = Path('/media/kevin/data_4t/Test')
file = path / 'Description_Images_summary.parquet'
df = pl.read_parquet(file)
print(df.shape, df['item_id'].n_unique())
df.head()

(200, 12) 200


item_id,item_name,shop_id,shop_name,level1_global_be_category,description,images,image_url,file_path,exists,img_info,text_info
i64,str,i64,str,str,str,str,str,str,bool,str,str
16693283776,"""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…",936501257,"""nailxinh.ling""","""Fashion Accessories""","""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…","""vn-11134201-23030-6scwoywxp9nv…","""http://f.shopee.vn/file/vn-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""N…","""```json { ""product_type"": ""N…"
17285376104,"""Váy 2 dây dài suông quốc dân S…",184210921,"""SUTANO FASHION""","""Women Clothes""","""[{""t"":""Váy 2 dây dài suông quố…","""sg-11134201-7rbmo-lqlqja37zjwk…","""http://f.shopee.vn/file/sg-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""D…","""```json { ""product_type"": ""D…"
25182695679,"""Set đồ nữ áo buộc cổ cúp ngực …",1287014463,"""TRENY_OFFICIAL""","""Women Clothes""","""Set đồ nữ áo buộc cổ cúp ngực …","""vn-11134207-7r98o-lwz6306v2p5l…","""http://f.shopee.vn/file/vn-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""D…","""```json { ""product_type"": ""D…"
15273431059,"""Quần jean nam DSQ xanh thêu lá…",682198966,"""KING77_FASHION""","""Men Clothes""","""[{""t"":""Chuỗi thời trang KING77…","""vn-11134207-7r98o-lsr2rxp5iax0…","""http://f.shopee.vn/file/vn-111…","""/media/kevin/data_4t/Test/down…",true,"""{ ""product_type"": ""Jeans"", …","""{ ""product_type"": ""Jeans"", …"
25130657466,"""Kẹp Tóc Nơ Màu Cô Gái Tam Giác…",803276458,"""Girlspower.vn""","""Fashion Accessories""","""💗 Chào mừng đến với cửa hàng c…","""sg-11134201-7rd66-lvb9le6gqnhs…","""http://f.shopee.vn/file/sg-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""H…","""```json { ""product_type"": ""H…"


In [3]:
df_q = PipelineText(mode='q').run(df)

INFO | run | [Data] Base Data db: (200, 12)


[Pipeline] Clean Text: 100%|██████████| 200/200 [00:00<00:00, 17303.59it/s]

INFO | run | [Data] Join Data db: (200, 13)
INFO | run | [Data] Base Data q: (200, 12)



[Pipeline] Clean Text: 100%|██████████| 200/200 [00:00<00:00, 22612.02it/s]

INFO | run | [Data] Join Data q: (200, 13)


In [4]:
df_q

q_item_id,q_item_name,q_shop_id,q_shop_name,q_level1_global_be_category,q_description,q_images,q_image_url,q_file_path,q_exists,q_img_info,q_text_info,q_item_name_clean
i64,str,i64,str,str,str,str,str,str,bool,str,str,str
16693283776,"""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…",936501257,"""nailxinh.ling""","""Fashion Accessories""","""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…","""vn-11134201-23030-6scwoywxp9nv…","""http://f.shopee.vn/file/vn-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""N…","""```json { ""product_type"": ""N…","""hộp 10 kẹp tóc nhỏ bằng nhựa b…"
17285376104,"""Váy 2 dây dài suông quốc dân S…",184210921,"""SUTANO FASHION""","""Women Clothes""","""[{""t"":""Váy 2 dây dài suông quố…","""sg-11134201-7rbmo-lqlqja37zjwk…","""http://f.shopee.vn/file/sg-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""D…","""```json { ""product_type"": ""D…","""váy 2 dây dài suông quốc dân s…"
25182695679,"""Set đồ nữ áo buộc cổ cúp ngực …",1287014463,"""TRENY_OFFICIAL""","""Women Clothes""","""Set đồ nữ áo buộc cổ cúp ngực …","""vn-11134207-7r98o-lwz6306v2p5l…","""http://f.shopee.vn/file/vn-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""D…","""```json { ""product_type"": ""D…","""set đồ nữ áo buộc cổ cúp ngực …"
15273431059,"""Quần jean nam DSQ xanh thêu lá…",682198966,"""KING77_FASHION""","""Men Clothes""","""[{""t"":""Chuỗi thời trang KING77…","""vn-11134207-7r98o-lsr2rxp5iax0…","""http://f.shopee.vn/file/vn-111…","""/media/kevin/data_4t/Test/down…",true,"""{ ""product_type"": ""Jeans"", …","""{ ""product_type"": ""Jeans"", …","""quần jean nam dsq xanh thêu lá…"
25130657466,"""Kẹp Tóc Nơ Màu Cô Gái Tam Giác…",803276458,"""Girlspower.vn""","""Fashion Accessories""","""💗 Chào mừng đến với cửa hàng c…","""sg-11134201-7rd66-lvb9le6gqnhs…","""http://f.shopee.vn/file/sg-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""H…","""```json { ""product_type"": ""H…","""kẹp tóc nơ màu cô gái tam giác…"
…,…,…,…,…,…,…,…,…,…,…,…,…
26161779574,"""[Mã FADEP2709 giảm đến 30k đơn…",30142553,"""NASU MAY -XƯỞNG MAY THỜI TRANG""","""Women Clothes""","""[{""t"":""Mô tả: Áo trễ vai kèm d…","""vn-11134207-7r98o-m0cy4y5x43xp…","""http://f.shopee.vn/file/vn-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""T…","""```json { ""product_type"": ""T…","""áo kiểu croptop trễ vai kèm dâ…"
22784930781,"""Vớ cotton Mềm Ngắn 5 Màu Trơn …",309936854,"""Baby Club""","""Baby & Kids Fashion""","""[{""t"":""🌟Mô tả sản phẩm🌟\nS: 0-…","""sg-11134201-7rblu-lne35gpvy1ui…","""http://f.shopee.vn/file/sg-111…","""/media/kevin/data_4t/Test/down…",true,"""{ ""product_type"": ""Socks"", …","""{ ""product_type"": ""Socks"", …","""vớ cotton mềm ngắn 5 màu trơn …"
24626852588,"""Áo Polo Nam Rap Lau Rừn Thêu…",504944462,"""Buffalo Boutique""","""Men Clothes""","""Áo Polo Nam Rap Lau Rừn Thêu…","""vn-11134207-7r98o-luofpopk0xlx…","""http://f.shopee.vn/file/vn-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""M…","""```json { ""product_type"": ""M…","""áo polo nam rap lau rừn thêu…"


In [5]:
path_tmp_array = Path('tmp/array')
path_tmp_ds = Path('tmp/ds')
make_dir(path_tmp_ds)
make_dir(path_tmp_array)

file_embed = path_tmp_array / 'embed_item_name.npy'
if not file_embed.exists():
    model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False)
    embeddings = model.encode(
        df_q['q_item_name_clean'].to_list(),
        batch_size=8,
        max_length=80,
        return_dense=True,
        return_sparse=False,
        return_colbert_vecs=False
    )['dense_vecs']
    np.save(file_embed, embeddings)
else:
    embeddings = np.load(file_embed)
print(embeddings.shape)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

/home/kevin/miniforge3/envs/item/lib/python3.12/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  colbert_state_dict = torch.

(200, 1024)


In [6]:
df_q = df_q.with_columns(pl.Series(values=embeddings, name='embed'))
dataset = Dataset.from_polars(df_q)
dataset.set_format(type='numpy', columns=['embed'], output_all_columns=True)

In [7]:
path_index = Path('tmp/index')
build_index(
    str(path_tmp_array),
    index_path=str(path_index / f'ip_item_name.index'),
    index_infos_path=str(path_index / f'index.json'),
    save_on_disk=True,
    metric_type='ip',
    verbose=30,
)

  0%|          | 0/1 [00:00<?, ?it/s]


(<faiss.swigfaiss_avx2.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *' at 0x79eb74f1bc60> >,
 {'index_key': 'Flat',
  'index_param': '',
  'index_path': 'tmp/index/ip_item_name.index',
  'size in bytes': 819245,
  'avg_search_speed_ms': 0.034973492482095025,
  '99p_search_speed_ms': 0.16855860965733882,
  'reconstruction error %': 0.0,
  'nb vectors': 200,
  'vectors dimension': 1024,
  'compression ratio': 0.9999450713766944})

In [9]:
# add index
dataset.load_faiss_index('embed', path_index / f'ip_item_name.index')

In [15]:
score, result = dataset.get_nearest_examples_batch(
    'embed',
    np.asarray(dataset['embed']),
    k=5
)

dict_ = {'score': [list(i) for i in score]}
df_score = pl.DataFrame(dict_)
df_result = (
    pl.DataFrame(result).drop(['embed'])
    .select(pl.all().name.prefix(f'db_'))
)

df_match = pl.concat([df_q.drop(['embed']), df_result, df_score], how='horizontal')
col_explode = [i for i in df_match.columns if 'db' in i] + ['score']
df_match = df_match.explode(col_explode)

In [16]:
df_match

q_item_id,q_item_name,q_shop_id,q_shop_name,q_level1_global_be_category,q_description,q_images,q_image_url,q_file_path,q_exists,q_img_info,q_text_info,q_item_name_clean,db_q_item_id,db_q_item_name,db_q_shop_id,db_q_shop_name,db_q_level1_global_be_category,db_q_description,db_q_images,db_q_image_url,db_q_file_path,db_q_exists,db_q_img_info,db_q_text_info,db_q_item_name_clean,score
i64,str,i64,str,str,str,str,str,str,bool,str,str,str,i64,str,i64,str,str,str,str,str,str,bool,str,str,str,f32
16693283776,"""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…",936501257,"""nailxinh.ling""","""Fashion Accessories""","""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…","""vn-11134201-23030-6scwoywxp9nv…","""http://f.shopee.vn/file/vn-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""N…","""```json { ""product_type"": ""N…","""hộp 10 kẹp tóc nhỏ bằng nhựa b…",16693283776,"""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…",936501257,"""nailxinh.ling""","""Fashion Accessories""","""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…","""vn-11134201-23030-6scwoywxp9nv…","""http://f.shopee.vn/file/vn-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""N…","""```json { ""product_type"": ""N…","""hộp 10 kẹp tóc nhỏ bằng nhựa b…",1.0
16693283776,"""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…",936501257,"""nailxinh.ling""","""Fashion Accessories""","""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…","""vn-11134201-23030-6scwoywxp9nv…","""http://f.shopee.vn/file/vn-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""N…","""```json { ""product_type"": ""N…","""hộp 10 kẹp tóc nhỏ bằng nhựa b…",23752982332,"""Haimeikang Set 10 Kẹp Tóc Nơ X…",263746294,"""Haimeikang.vn""","""Fashion Accessories""","""[{""i"":""sg-11134202-7rbk0-ll1w2…","""sg-11134201-7rbk0-ll1w2fzayxaw…","""http://f.shopee.vn/file/sg-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""H…","""```json { ""product_type"": ""H…","""haimeikang set 10 kẹp tóc nơ x…",0.669004
16693283776,"""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…",936501257,"""nailxinh.ling""","""Fashion Accessories""","""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…","""vn-11134201-23030-6scwoywxp9nv…","""http://f.shopee.vn/file/vn-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""N…","""```json { ""product_type"": ""N…","""hộp 10 kẹp tóc nhỏ bằng nhựa b…",23788633578,"""Kẹp Đầu Nhỏ Đính Ngọc Trai Siê…",1132437531,"""Phụ kiện tóc Yifen bán buôn""","""Fashion Accessories""","""🧨🧨🧨Chào mừng đến với Yifen Sto…","""cn-11134207-7r98o-ls5h7vy2mvh9…","""http://f.shopee.vn/file/cn-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""H…","""```json { ""product_type"": ""H…","""kẹp đầu nhỏ đính ngọc trai siê…",0.635674
16693283776,"""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…",936501257,"""nailxinh.ling""","""Fashion Accessories""","""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…","""vn-11134201-23030-6scwoywxp9nv…","""http://f.shopee.vn/file/vn-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""N…","""```json { ""product_type"": ""N…","""hộp 10 kẹp tóc nhỏ bằng nhựa b…",21123185268,"""Kẹp Tóc Hình Nơ Đính Ngọc Trai…",405832779,"""Hair accessories.vn""","""Fashion Accessories""","""[{""t"":""Thời gian giao hàng ước…","""e98aeb750219738b8fa4a53546cbf3…","""http://f.shopee.vn/file/e98aeb…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""H…","""```json { ""product_type"": ""H…","""kẹp tóc hình nơ đính ngọc trai…",0.614015
16693283776,"""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…",936501257,"""nailxinh.ling""","""Fashion Accessories""","""Hộp 10 Kẹp Tóc Nhỏ Bằng Nhựa B…","""vn-11134201-23030-6scwoywxp9nv…","""http://f.shopee.vn/file/vn-111…","""/media/kevin/data_4t/Test/down…",true,"""```json { ""product_type"": ""N…","""```json { ""product_type"": ""N…","""hộp 10 kẹp tóc nhỏ bằng nhựa b…",17777258614,"""Kẹp tóc kiểu hoa tulip 2 đầu t…",178706384,"""lovelyhome005.vn""","""Fashion Accessories""","""[{""t"":""Thời gian giao hàng dự …","""d1780cd92fdf820141c90d940abb2a…",